# Download Data
Download the data used for this study from Petrel

In [1]:
from globus_sdk import TransferData
from mdf_forge import Forge
import os

Locations of data to be used in study

In [2]:
source_ep = 'e38ee745-6d04-11e5-ba46-22000b92c6ec'
source_paths = [os.path.join('Schleife', f) for f in [
    '256_Al', 'Al_256_channel', 'Al_256_new'
]]

Where to save the data

In [3]:
with open(os.path.join(os.path.expanduser('~'), '.globusonline', 'lta', 'client-id.txt')) as fp:
    dest_ep = fp.readline()
dest_path = os.path.abspath('datasets')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mrl-mrsec/.globusonline/lta/client-id.txt'

## Get a Transfer Client
Use Forge from the MDF Client to create a transfer client. Forge contains the logic for retrieving and saving the security tokens needed by Globus

In [5]:
forge = Forge()

In [6]:
transfer_client = forge.transfer_client

## Initiate the Transfer
Set up the transfer by defining the data to copy and where to put it

In [7]:
tdata = TransferData(transfer_client, source_ep, dest_ep)

In [8]:
for data in source_paths:
    tdata.add_item(data, 
                   os.path.join(dest_path, os.path.basename(data)), recursive=True)

In [9]:
tdata

{'DATA_TYPE': 'transfer',
 'submission_id': 'a151b911-b139-11e8-823e-0a3b7ca8ce66',
 'source_endpoint': 'e38ee745-6d04-11e5-ba46-22000b92c6ec',
 'destination_endpoint': '30b7bcac-b136-11e8-823e-0a3b7ca8ce66\n',
 'verify_checksum': False,
 'preserve_timestamp': False,
 'encrypt_data': False,
 'recursive_symlinks': 'ignore',
 'DATA': [{'DATA_TYPE': 'transfer_item',
   'source_path': 'Schleife/256_Al',
   'destination_path': '/home/wardlt/Schleife-Stopping-Power/datasets/256_Al',
   'recursive': True},
  {'DATA_TYPE': 'transfer_item',
   'source_path': 'Schleife/Al_256_channel',
   'destination_path': '/home/wardlt/Schleife-Stopping-Power/datasets/Al_256_channel',
   'recursive': True},
  {'DATA_TYPE': 'transfer_item',
   'source_path': 'Schleife/Al_256_new',
   'destination_path': '/home/wardlt/Schleife-Stopping-Power/datasets/Al_256_new',
   'recursive': True}]}

Submit the transfer

In [10]:
trans_id = transfer_client.submit_transfer(tdata)

Wait for it to finish

In [13]:
trans_id

TransferResponse({'DATA_TYPE': 'transfer_result', 'code': 'Accepted', 'message': 'The transfer has been accepted and a task has been created and queued for execution', 'request_id': 'Y9zrJRcRv', 'resource': '/transfer', 'submission_id': 'a151b911-b139-11e8-823e-0a3b7ca8ce66', 'task_id': 'a151b910-b139-11e8-823e-0a3b7ca8ce66', 'task_link': {'DATA_TYPE': 'link', 'href': 'task/a151b910-b139-11e8-823e-0a3b7ca8ce66?format=json', 'rel': 'related', 'resource': 'task', 'title': 'related task'}})

In [15]:
while not transfer_client.task_wait(trans_id['task_id']):
    tstatus = transfer_client.get_task(trans_id['task_id'])
    print('\rTransfer status: {} - Fraction complete: {:.2f}%'.format(tstatus['status'], tstatus['subtasks_succeeded'] / tstatus['subtasks_total'] * 100), 
         end="")